In [82]:
import torch
import torchvision
from models.inception_resnet_v1 import InceptionResnetV1

checkpoint = torch.load(
        "checkpoints/charmed-cosmos-135_epoch_19",
        # "checkpoints/major-cloud-212_epoch_19",
        map_location=torch.device("cpu")
)
model = InceptionResnetV1()
model.load_state_dict(checkpoint["model_state_dict"])
model.eval()
print(".")

.


In [ ]:
if torch.cuda.is_available():
    model.cuda()

In [68]:
#dataset = WebfaceDataset("../../data/Aligned_CASIA_WebFace")
from data.data_loaders import get_data_loaders
from data.web_face_dataset import WebfaceDataset
torch.manual_seed(42)
dataset = WebfaceDataset("datasets/Aligned_CASIA_WebFace")
CLASSES_PER_BATCH = 35
SAMPLES_PER_CLASS = 40
BATCH_SIZE = CLASSES_PER_BATCH * SAMPLES_PER_CLASS

train_loader, val_loader, test_loader = get_data_loaders(
    dataset,
    CLASSES_PER_BATCH,
    SAMPLES_PER_CLASS,
    train_proportion=0.8,
    val_proportion=0.1,
    test_proportion=0.1,
)

In [51]:

# from classifier.faiss_classifier import FaissClassifier
# from sklearn.metrics import accuracy_score, f1_score
# from tqdm import tqdm
# import numpy as np

# def evaluate(model, val_loader, train_loader):


#     classifier = FaissClassifier()
#     classifier.threshold = 0.02 

#     accuracy = 0
#     f1 = 0
#     count = 0

#     with torch.no_grad():
#         model.eval()

#         for _, (data, target) in tqdm(
#             enumerate(val_loader), total=len(val_loader), desc="evaluating batch: "
#         ):
            
#             if torch.cuda.is_available():
#                 data = data.cuda()
#                 target = target.cuda()

#             outputs = model(data)
#             predicted = classifier.classify(outputs.data.cpu().numpy())
            
#             target = target.cpu()
#             print("target:", target.numpy(), "Predicted:", np.array([predicted]))
#             accuracy += accuracy_score(target.numpy(), np.array([predicted]))
#             f1 += f1_score(target.numpy(), np.array([predicted]), average="weighted")
#             count += 1

#     total_accuracy = accuracy / count
#     total_f1 = np.array(f1) / count

#     print(total_accuracy, accuracy, count)
#     return total_accuracy, total_f1

In [52]:
# total_accuracy, total_f1 = evaluate(model, val_loader, train_loader)
# print("Accuracy:",total_accuracy)
# print("F1 score:", total_f1 )

t: [318] Predicted: [318]
evaluating batch: 100%|██████████| 794/794 [05:00<00:00,  2.64it/s]target: [260] Predicted: [1246238]
0.8060453400503779 640.0 794
Accuracy: 0.8060453400503779
F1 score: 0.8060453400503779



In [79]:
# total_accuracy, total_f1 = evaluate(model, val_loader, train_loader)
# print("Accuracy:",total_accuracy)
# print("F1 score:", total_f1 )

evaluating batch: 100%|██████████| 794/794 [04:39<00:00,  2.84it/s]target: [301] Predicted: [301]
0.7858942065491183 624.0 794
Accuracy: 0.7858942065491183
F1 score: 0.7858942065491183



In [77]:
def evaluate(model, val_loader, train_loader):


    classifier = FaissClassifier(index = "datasets/vector_pre_trained.index")
    classifier.threshold = 0.02 


    true = []
    pred = []

    with torch.no_grad():
        model.eval()

        for _, (data, target) in tqdm(
            enumerate(val_loader), total=len(val_loader), desc="evaluating batch: "
        ):
            
            if torch.cuda.is_available():
                data = data.cuda()
                target = target.cuda()

            outputs = model(data)
            predicted = classifier.classify(outputs.data.cpu().numpy())
            
            target = target.cpu()
            print("target:", target.numpy(), "Predicted:", np.array([predicted]))

            # decided to put this one in list so that F1 score can be calculated
            true.append(int(target.numpy()))
            pred.append(predicted)
            

    total_accuracy = accuracy_score(np.array(true), np.array(pred))
    total_f1 = f1_score(np.array(true), np.array(pred), average="weighted")

    print(total_accuracy)
    return total_accuracy, total_f1

In [80]:
total_accuracy, total_f1 = evaluate(model, val_loader, train_loader)
print("Accuracy:",total_accuracy)
print("F1 score:", total_f1 )

2it/s]target: [233] Predicted: [5231]
evaluating batch: 100%|██████████| 794/794 [04:43<00:00,  2.80it/s]target: [301] Predicted: [301]
[204, 210, 324, 321, 225, 295, 195, 318, 299, 321, 254, 205, 198, 271, 202, 268, 301, 302, 225, 210, 210, 301, 324, 198, 202, 198, 249, 295, 198, 275, 331, 260, 207, 335, 205, 233, 275, 202, 301, 332, 268, 195, 210, 302, 204, 307, 195, 301, 225, 198, 286, 275, 195, 225, 333, 247, 318, 260, 233, 284, 204, 318, 333, 207, 333, 211, 240, 220, 225, 333, 207, 207, 195, 195, 202, 195, 302, 302, 204, 281, 302, 307, 238, 204, 318, 270, 293, 284, 212, 260, 301, 307, 286, 208, 263, 204, 263, 280, 195, 295, 318, 307, 210, 307, 295, 295, 210, 202, 233, 208, 195, 293, 286, 211, 275, 332, 204, 295, 333, 210, 204, 297, 195, 270, 210, 295, 287, 205, 295, 195, 202, 295, 332, 238, 233, 295, 301, 295, 204, 302, 295, 270, 293, 268, 210, 332, 318, 335, 202, 263, 207, 207, 335, 204, 332, 205, 307, 225, 260, 295, 302, 210, 301, 233, 195, 195, 307, 275, 295, 205, 281, 225, 212